In [1]:
from pyspark import SparkContext
sc =SparkContext()

In [2]:
#讓我們在交互式會話中讀取一個文件。我們將在這裡從spark文件夾中讀取“CHANGES.txt”文件
RDDread = sc.textFile ("file:///home/hadoop/CHANGES.txt")

In [4]:
#它看起來怎樣？讓我們使用collect（）動作查看RDD的內容
RDDread. collect ()

['Spark Change Log',
 '----------------',
 '',
 'Release 1.1.0',
 '',
 '  [SPARK-3320][SQL] Made batched in-memory column buffer building work for SchemaRDDs with empty partitions',
 '  Cheng Lian <lian.cs.zju@gmail.com>',
 '  2014-08-29 18:16:47 -0700',
 '  Commit: aa9364a, github.com/apache/spark/pull/2213',
 '',
 '  [SPARK-3296][mllib] spark-example should be run-example in head notation of DenseKMeans and SparseNaiveBayes',
 '  wangfei <wangfei_hello@126.com>',
 '  2014-08-29 17:37:15 -0700',
 '  Commit: b0facb5, github.com/apache/spark/pull/2193',
 '',
 '  [SPARK-3291][SQL]TestcaseName in createQueryTest should not contain ":"',
 '  qiping.lqp <qiping.lqp@alibaba-inc.com>',
 '  2014-08-29 15:37:43 -0700',
 '  Commit: c1333b8, github.com/apache/spark/pull/2191',
 '',
 '  [SPARK-3269][SQL] Decreases initial buffer size for row set to prevent OOM',
 '  Cheng Lian <lian.cs.zju@gmail.com>',
 '  2014-08-29 15:36:04 -0700',
 '  Commit: 9bae345, github.com/apache/spark/pull/2171',
 '',
 '

In [5]:
#為了以有組織的格式顯示Spark RDD的內容，可以使用諸如“first（）”，“take（）”和“takeSample（False，10,2）”之類的動作。

#first（） -這將返回數據集中的第一個元素 顯示changes.txt文件的第一行
RDDread. first ()

'Spark Change Log'

In [6]:
#Take（n） - 這將返回數據集中的前n行，並在控制台上顯示它們
RDDread. take (5)

['Spark Change Log', '----------------', '', 'Release 1.1.0', '']

In [10]:
#TakeSample（withReplacemen t，n，[seed]） - 
#此操作將返回數據集中的n個元素，有或沒有替換（true或false）。Seed是一個可選參數，用作隨機生成器

RDDread. takeSample (False, 10, 2)

['  Merge pull request #530 from mbautin/master-update-log4j-and-make-compile-in-IntelliJ',
 '  Xiangrui Meng <meng@databricks.com>',
 '  Commit: d666053, github.com/apache/spark/pull/261',
 '  [SPARK-2659][SQL] Fix division semantics for hive',
 '  ca44b51 Tue Nov 5 01:32:55 2013 -0800',
 '',
 '  Commit: 71fcd2e, github.com/apache/spark/pull/1786',
 '',
 '  Reza Zadeh <rizlar@gmail.com>',
 '']

![title](picture.jpg)

In [16]:
#takeSample()函数和sample函数是一个原理，但是不使用相对比例采样，而是按设定的采样个数进行采样，同时返回结果不再是RDD，
#而是相当于对采样后的数据进行collect()，返回结果的集合为单机的数组。
#图中，左侧的方框代表分布式的各个节点上的分区，右侧方框代表单机上返回的结果数组。通过takeSample对数据采样，设置为采样一份数据，
#返回结果为V1

In [17]:
#count（） - 知道RDD中的行數
RDDread. count ()

14577

In [18]:
#RDD分區
# 並行性是任何分佈式系統的關鍵特性，其中操作通過將數據劃分為多個並行分區來完成。同時對分區執行相同的操作，這有助於利用spark實現快速數據處理
# 。通過將數據劃分為多個分區，可以在apache spark中並行地有效地應用Map和Reduce操作。RDD中每個分區的副本分佈在群集的不同節點上運行的多個工
# 作線程中，以便在單個工作程序發生故障時RDD仍然可用。

# RDD上每個操作的並行度取決於RDD具有的固定分區數。我們可以使用repartition（）和coalesce（）方法在創建或稍後指定並行度或分區數

partRDD = sc.textFile("file:///home/hadoop/CHANGES.txt",4)


In [19]:
#coalesce（）是repartition（）方法的優化版本，可避免數據移動，通常用於在過濾大型數據集後減少分區數。

#可以使用以下方法檢查RDD當前的分區數 - rdd.getNumPartitions（）
partRDD.getNumPartitions()

4

In [20]:
# 使用reduceByKey操作處理數據時，Spark將根據默認並行性形成多個輸出分區，這取決於每個節點上可用的節點數和內核數。

# 以下是兩個版本的地圖轉換，它們分別利用RDD的每個分區，分別利用火花集群的最大核心和內存 -

# partRDD.mapPartitions（）：這在每個分區上單獨運行一個map操作，這與普通的map操作不同，在map操作中map用於對整個RDD的每一行進行操作。

# mapPartitionsWithIndex（）：這與partRDD.mapPartitions的作用相同，但我們還可以指定必須應用此操作的分區號。

In [22]:
# UDF（用戶定義的函數）

# UDF提供了一種向Spark添加單獨函數的簡單方法，可以在各種轉換階段使用。UDF通常用於在Spark RDD上執行多個任務。

# 讓我們用一個簡單的用例來理解使用movie數據集的上述概念。

# 關於數據集：

# u.user - 有關用戶的人口統計信息; 這是一個以製表符分隔的列表
userRDD = sc.textFile("file:///home/hadoop//u.user")

In [23]:
userRDD.count()

943

In [24]:
# 創建Spark應用程序

# 讓我們創建一個用戶定義的函數來將用戶劃分為年齡組
def parse_N_calculate_age(data):
             userid,age,gender,occupation,zip = data.split("|")
             return  userid, age_group(int(age)),gender,occupation,zip,int(age)
def  age_group(age):
        if age < 10 :
           return '0-10'
        elif age < 20:
           return '10-20'
        elif age < 30:
           return '20-30'
        elif age < 40:
           return '30-40'
        elif age < 50:
           return '40-50'
        elif age < 60:
           return '50-60'
        elif age < 70:
           return '60-70'
        elif age < 80:
           return '70-80'
        else :
           return '80+'

In [25]:
data_with_age_bucket = userRDD.map(parse_N_calculate_age)

In [27]:
#現在，讓我們分析年齡組“20-30”進行進一步分析
RDD_20_30 = data_with_age_bucket.filter(lambda line : '20-30' in line)

In [28]:
# 由於我們要分析20-30歲年齡組的多個事物，我們可以將它放在內存中以便進行這些操作，這樣就可以花更少的時間進行計算。

# 讓我們根據他們在給定age_group 20-30中的職業來計算用戶數量
freq = RDD_20_30.map(lambda line : line[3]).countByValue()

dict(freq)

{'administrator': 19,
 'artist': 12,
 'doctor': 2,
 'educator': 12,
 'engineer': 23,
 'entertainment': 8,
 'executive': 7,
 'healthcare': 4,
 'homemaker': 3,
 'lawyer': 4,
 'librarian': 11,
 'marketing': 5,
 'none': 2,
 'other': 38,
 'programmer': 30,
 'salesman': 2,
 'scientist': 8,
 'student': 116,
 'technician': 12,
 'writer': 14}

In [29]:
#現在讓我們根據性別計算同一年齡組的電影用戶數量
age_wise = RDD_20_30.map (lambda line : line[2]).countByValue()

dict(age_wise)

{'F': 85, 'M': 247}

In [30]:
#由於我們完成了對上述緩存數據的操作，我們可以使用unpersisit（）方法將它們從內存中刪除 -
RDD_20_30.unpersist()

PythonRDD[18] at RDD at PythonRDD.scala:48

In [31]:
# 現在，我們將使用累加器在上述電影數據集中進行異常值檢測。讓我們假設任何年齡組80歲以上的人都是異常值並被標記為過度使用，任何屬於0-10歲年
# 齡段的人都是異常值並標記為不足。
Under_age = sc.accumulator(0)

Over_age = sc.accumulator(0)

In [32]:
def outliers(data):
    global Over_age, Under_age
    age_grp = data[1]
    if(age_grp == "70-80"):
        Over_age +=1
    if(age_grp == "0-10"):
        Under_age +=1
    return data

In [33]:
#使用上面的方法通過一個用於計算異常值的函數傳遞整個RDD
df = data_with_age_bucket.map(outliers).collect()

In [34]:
#現在我們將檢查有多少用戶未成年以及有多少人超過年齡 -
Under_age.value

1

In [35]:
Over_age.value

4

In [38]:
#完整程式
userRDD = sc.textFile("file:///home/hadoop//u.user")

def parse_N_calculate_age(data):
             userid,age,gender,occupation,zip = data.split("|")
             return  userid, age_group(int(age)),gender,occupation,zip,int(age)

def  age_group(age):
        if age < 10 :
           return '0-10'
        elif age < 20:
           return '10-20'
        elif age < 30:
           return '20-30'
        elif age < 40:
           return '30-40'
        elif age < 50:
           return '40-50'
        elif age < 60:
           return '50-60'
        elif age < 70:
           return '60-70'
        elif age < 80:
           return '70-80'
        else :
           return '80+'

data_with_age_bucket = userRDD.map(parse_N_calculate_age)

RDD_20_30 = data_with_age_bucket.filter(lambda line : '20-30' in line)

freq = RDD_20_30.map(lambda line : line[3]).countByValue()

print ("total user count is ",userRDD.count())

print ("total movie users profession wise ",dict(freq))

Under_age = sc.accumulator(0)
Over_age = sc.accumulator(0)

def outliers(data):
    global Over_age, Under_age
    age_grp = data[1]
    if(age_grp == "70-80"):
        Over_age +=1
    if(age_grp == "0-10"):
        Under_age +=1
    return data

df = data_with_age_bucket.map(outliers).collect()

print ("under age users of the movie are ",Under_age)
print ("over age users of the movie are ",Over_age)

total user count is  943
total movie users profession wise  {'marketing': 5, 'homemaker': 3, 'other': 38, 'educator': 12, 'engineer': 23, 'executive': 7, 'none': 2, 'salesman': 2, 'lawyer': 4, 'librarian': 11, 'administrator': 19, 'artist': 12, 'technician': 12, 'entertainment': 8, 'healthcare': 4, 'programmer': 30, 'student': 116, 'writer': 14, 'doctor': 2, 'scientist': 8}
under age users of the movie are  1
over age users of the movie are  4
